In [1]:
import tensorflow as tf
from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
import wandb
from wandb.keras import WandbCallback

2023-06-05 10:11:00.919767: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 10:11:01.739777: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
project_name = "transforming-prosit"

In [3]:
run = wandb.init(project=project_name)

wandb: Currently logged in as: mamisashvili-lizi. Use `wandb login --relogin` to force relogin


In [5]:
model_artifact_name = "mamisashvili-lizi/transforming-prosit/model-transformers_v2:v0"

In [6]:
model = run.use_artifact(model_artifact_name)

In [6]:
model_dir = model.download()

wandb:   5 of 5 files downloaded.  


In [17]:
model_dir

'./artifacts/model-transformers_v2:v0'

In [1]:
from dlomix.data import IntensityDataset

TRAIN_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/main/example_datasets/Intensity/proteomeTools_train_val.csv'
BATCH_SIZE = 64
int_data = IntensityDataset(data_source=TRAIN_DATAPATH, seq_length=30,
                            collision_energy_col='collision_energy', batch_size=BATCH_SIZE, val_ratio=0.2, test=False)

2023-06-09 08:32:16.359177: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 08:32:16.605682: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-09 08:32:17.490745: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-09 08:32:17.490977: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [5]:
"Training examples", BATCH_SIZE * len(int_data.train_data), "Validation examples", BATCH_SIZE * len(int_data.val_data)

('Training examples', 23616, 'Validation examples', 5952)

In [19]:
model = tf.keras.models.load_model(model_dir, custom_objects={
    "masked_spectral_distance": masked_spectral_distance,
    "masked_pearson_correlation_distance": masked_pearson_correlation_distance
})

In [20]:
model.summary()

Model: "prosit_transformer_intensity_predictor_v2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 meta_encoder (MetaEncoder)  multiple                  128       
                                                                 
 string_lookup (StringLookup  multiple                 0         
 )                                                               
                                                                 
 embedding (Embedding)       multiple                  352       
                                                                 
 encoder_att (AttentionLayer  multiple                 46        
 )                                                               
                                                                 
 fusion_layer (FusionLayer)  multiple                  0         
                                                                 
 sequence_encoder_no_gru 